In [ ]:
# DNN_A with 1025 input and 2050 output
# DNN_C with 2050 input and 2050 output
# Half of the data is used to train DNN_A and the other half is given to trained DNN_A to generate data for DNN_C
# DNN_C has it's own cost function(customized)

In [1]:
#import libraries.
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.python.keras.utils.data_utils import Sequence
# from keras.models import Sequential
# from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
import keras
from keras import regularizers
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
import sounddevice as sd
import tensorflow as tf
from mir_eval import separation 
from pystoi.stoi import stoi 

Using TensorFlow backend.


In [2]:
#import libraries.
import matplotlib.pyplot as plt
# from tabulate import tabulate
import time
import os
import librosa
from librosa.core import stft, istft
import time
import pickle
from keras import backend as K
from tensorflow.keras.models import load_model
# from keras.models import load_model

In [3]:
#read necessary files, contarins mixed and clean features.
def data_reading(i):
    temp = open( "TIMIT/Organized/concatenated/Mixed" +str(i) + ".p", "rb")
    ftr = pickle.load( temp  ) .T
    temp.close()
    #contarins target(clean) log power features.
    temp = open( "TIMIT/Organized/concatenated/clean" +str(i) + ".p", "rb")
    target = pickle.load( temp ).T
    temp.close()
    return ftr, target

In [4]:
def phase_reading(i):
    temp = open( "TIMIT/Organized/concatenated/Mixed_phase" +str(i) + ".p", "rb")
    angle = pickle.load( temp  ) .T
    temp.close()
    #contarins target(clean) log power features.
    return angle

In [6]:
ftr , target = data_reading(0)

In [8]:
X = ftr
y = target

In [5]:
h_A = [1025,1025,1025]
h_C = [4100, 4100, 4100]

In [10]:
ftr = []
target = []

In [6]:
def DNN_C_loss(y_true,y_pred):
    c = 0.2 # c is equal to lambda in the paper
    term1 = K.sum(((y_true - y_pred)**2))
    term2 = K.sum((y_true[0:1025] - y_pred[1025:2050])**2) + K.sum((y_true[1025:2050] - y_pred[0:1025])**2)
    loss = term1 - c * term2
    return loss

In [6]:
def DNN_A():
        # create model
        model = Sequential()
        model.add(Dense(h_A[0], input_dim = X.shape[1], kernel_initializer='normal', activation='relu'))
        model.add(Dense(h_A[1], kernel_initializer='normal', activation='relu'))
        model.add(Dense(h_A[2], kernel_initializer='normal', activation='relu'))
        model.add(Dense(y.shape[1], kernel_initializer='normal'))
        # Compile model
        model.compile(loss='mean_squared_error', optimizer='adam')
        return model

In [8]:
def DNN_C():
        # create model
        model = Sequential()
        model.add(Dense(h_C[0], input_dim = X.shape[1], kernel_initializer='normal', activation='relu'))
        model.add(Dense(h_C[1], kernel_initializer='normal', activation='relu'))
        model.add(Dense(h_C[2], kernel_initializer='normal', activation='relu'))
        model.add(Dense(y.shape[1], kernel_initializer='normal'))
        # Compile model
        model.compile(loss=DNN_C_loss, optimizer='adam')
        return model

In [6]:
def reconstruct(wave,angle):
    recon1 = wave*np.cos(angle)+wave*np.sin(angle)*1j
#     recon = np.sqrt(np.power(10, wave))
#     recon1 = recon*np.cos(angle)+recon*np.sin(angle)*1j
    recon = librosa.core.istft((recon1.T), hop_length=512, win_length=2048, window='hann')
    return recon

In [14]:
# seed = 1
# np.random.seed(seed)
print("^^^^^^^^^^^^^^^^^^^^^ Training with 5-fold on mixed0^^^^^^^^^^^^^^^^^^^^^")
estimator_A = KerasRegressor(build_fn=DNN_A, epochs=50, batch_size=150, verbose=1)
# kfold = KFold(n_splits=2, random_state=seed)
kfold = KFold(n_splits=5)
results = cross_val_score(estimator_A, X, y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))
print("^^^^^^^^^^^^^^^^^^^^^ Training on mixed0^^^^^^^^^^^^^^^^^^^^^")
estimator_A.fit(X, y)
# prediction = estimator_A.predict(X_test)

^^^^^^^^^^^^^^^^^^^^^ Training with 5-fold on mixed0^^^^^^^^^^^^^^^^^^^^^
Epoch 1/1000
93080/93080 [==============================] - 12s 129us/step - loss: 0.1649
Epoch 2/1000
93080/93080 [==============================] - 4s 46us/step - loss: 0.1294
Epoch 3/1000
93080/93080 [==============================] - 4s 46us/step - loss: 0.1231
Epoch 4/1000
93080/93080 [==============================] - 4s 46us/step - loss: 0.1197
Epoch 5/1000
93080/93080 [==============================] - 4s 46us/step - loss: 0.1166
Epoch 6/1000
93080/93080 [==============================] - 4s 47us/step - loss: 0.1134
Epoch 7/1000
93080/93080 [==============================] - 4s 46us/step - loss: 0.1099
Epoch 8/1000
93080/93080 [==============================] - 4s 47us/step - loss: 0.1046
Epoch 9/1000
93080/93080 [==============================] - 4s 47us/step - loss: 0.0989
Epoch 10/1000
93080/93080 [==============================] - 4s 47us/step - loss: 0.0938
Epoch 11/1000
93080/93080 [================

93080/93080 [==============================] - 4s 47us/step - loss: 0.0260
Epoch 184/1000
93080/93080 [==============================] - 4s 46us/step - loss: 0.0272
Epoch 185/1000
93080/93080 [==============================] - 4s 46us/step - loss: 0.0272
Epoch 186/1000
93080/93080 [==============================] - 4s 47us/step - loss: 0.0261
Epoch 187/1000
93080/93080 [==============================] - 4s 46us/step - loss: 0.0261
Epoch 188/1000
93080/93080 [==============================] - 4s 46us/step - loss: 0.0260
Epoch 189/1000
93080/93080 [==============================] - 4s 46us/step - loss: 0.0268
Epoch 190/1000
93080/93080 [==============================] - 4s 47us/step - loss: 0.0265
Epoch 191/1000
93080/93080 [==============================] - 4s 46us/step - loss: 0.0276
Epoch 192/1000
93080/93080 [==============================] - 4s 46us/step - loss: 0.0271
Epoch 193/1000
93080/93080 [==============================] - 4s 46us/step - loss: 0.0279
Epoch 194/1000
93080/9308

93080/93080 [==============================] - 4s 46us/step - loss: 0.0242
Epoch 366/1000
93080/93080 [==============================] - 4s 47us/step - loss: 0.0244
Epoch 367/1000
93080/93080 [==============================] - 4s 46us/step - loss: 0.0247
Epoch 368/1000
93080/93080 [==============================] - 4s 46us/step - loss: 0.0239
Epoch 369/1000
93080/93080 [==============================] - 4s 46us/step - loss: 0.0236
Epoch 370/1000
93080/93080 [==============================] - 4s 47us/step - loss: 0.0244
Epoch 371/1000
93080/93080 [==============================] - 4s 46us/step - loss: 0.0242
Epoch 372/1000
93080/93080 [==============================] - 4s 46us/step - loss: 0.0234
Epoch 373/1000
93080/93080 [==============================] - 4s 47us/step - loss: 0.0236
Epoch 374/1000
93080/93080 [==============================] - 4s 46us/step - loss: 0.0267
Epoch 375/1000
93080/93080 [==============================] - 4s 46us/step - loss: 0.0240
Epoch 376/1000
93080/9308

93080/93080 [==============================] - 4s 46us/step - loss: 0.0235
Epoch 548/1000
93080/93080 [==============================] - 4s 47us/step - loss: 0.0233
Epoch 549/1000
93080/93080 [==============================] - 4s 46us/step - loss: 0.0242
Epoch 550/1000
93080/93080 [==============================] - 4s 46us/step - loss: 0.0239
Epoch 551/1000
93080/93080 [==============================] - 4s 46us/step - loss: 0.0235
Epoch 552/1000
93080/93080 [==============================] - 4s 47us/step - loss: 0.0248
Epoch 553/1000
93080/93080 [==============================] - 4s 46us/step - loss: 0.0239
Epoch 554/1000
93080/93080 [==============================] - 4s 46us/step - loss: 0.0228
Epoch 555/1000
93080/93080 [==============================] - 4s 47us/step - loss: 0.0237
Epoch 556/1000
93080/93080 [==============================] - 4s 46us/step - loss: 0.0229
Epoch 557/1000
93080/93080 [==============================] - 4s 46us/step - loss: 0.0231
Epoch 558/1000
93080/9308

93080/93080 [==============================] - 4s 47us/step - loss: 0.0226
Epoch 730/1000
93080/93080 [==============================] - 4s 46us/step - loss: 0.0238
Epoch 731/1000
93080/93080 [==============================] - 4s 46us/step - loss: 0.0250
Epoch 732/1000
93080/93080 [==============================] - 4s 47us/step - loss: 0.0237
Epoch 733/1000
93080/93080 [==============================] - 4s 46us/step - loss: 0.0241
Epoch 734/1000
93080/93080 [==============================] - 4s 46us/step - loss: 0.0228
Epoch 735/1000
93080/93080 [==============================] - 4s 47us/step - loss: 0.0236
Epoch 736/1000
93080/93080 [==============================] - 4s 46us/step - loss: 0.0229
Epoch 737/1000
93080/93080 [==============================] - 4s 46us/step - loss: 0.0229
Epoch 738/1000
93080/93080 [==============================] - 4s 46us/step - loss: 0.0232
Epoch 739/1000
93080/93080 [==============================] - 4s 46us/step - loss: 0.0232
Epoch 740/1000
93080/9308

...

116351/116351 [==============================] - 13s 114us/step - loss: 0.0272
Epoch 265/1000
116351/116351 [==============================] - 13s 114us/step - loss: 0.0261
Epoch 266/1000
116351/116351 [==============================] - 13s 113us/step - loss: 0.0267
Epoch 267/1000
116351/116351 [==============================] - 13s 114us/step - loss: 0.0264
Epoch 268/1000
116351/116351 [==============================] - 13s 113us/step - loss: 0.0281
Epoch 269/1000
116351/116351 [==============================] - 13s 114us/step - loss: 0.0283
Epoch 270/1000
116351/116351 [==============================] - 13s 113us/step - loss: 0.0259
Epoch 271/1000
116351/116351 [==============================] - 13s 114us/step - loss: 0.0255
Epoch 272/1000
116351/116351 [==============================] - 13s 114us/step - loss: 0.0256
Epoch 273/1000
116351/116351 [==============================] - 13s 114us/step - loss: 0.0270
Epoch 274/1000
116351/116351 [==============================] - 13s 114us/s

116351/116351 [==============================] - 13s 110us/step - loss: 0.0253
Epoch 439/1000
116351/116351 [==============================] - 13s 110us/step - loss: 0.0251
Epoch 440/1000
116351/116351 [==============================] - 13s 110us/step - loss: 0.0249
Epoch 441/1000
116351/116351 [==============================] - 13s 109us/step - loss: 0.0244
Epoch 442/1000
116351/116351 [==============================] - 13s 110us/step - loss: 0.0253
Epoch 443/1000
116351/116351 [==============================] - 13s 109us/step - loss: 0.0252
Epoch 444/1000
116351/116351 [==============================] - 13s 110us/step - loss: 0.0259
Epoch 445/1000
116351/116351 [==============================] - 13s 109us/step - loss: 0.0258
Epoch 446/1000
116351/116351 [==============================] - 13s 110us/step - loss: 0.0253
Epoch 447/1000
116351/116351 [==============================] - 13s 110us/step - loss: 0.0249
Epoch 448/1000
116351/116351 [==============================] - 13s 110us/s

116351/116351 [==============================] - 13s 110us/step - loss: 0.0266
Epoch 613/1000
116351/116351 [==============================] - 13s 109us/step - loss: 0.0243
Epoch 614/1000
116351/116351 [==============================] - 13s 110us/step - loss: 0.0246
Epoch 615/1000
116351/116351 [==============================] - 13s 109us/step - loss: 0.0244
Epoch 616/1000
116351/116351 [==============================] - 13s 110us/step - loss: 0.0242
Epoch 617/1000
116351/116351 [==============================] - 13s 110us/step - loss: 0.0245
Epoch 618/1000
116351/116351 [==============================] - 13s 110us/step - loss: 0.0251
Epoch 619/1000
116351/116351 [==============================] - 13s 110us/step - loss: 0.0247
Epoch 620/1000
116351/116351 [==============================] - 13s 110us/step - loss: 0.0241
Epoch 621/1000
116351/116351 [==============================] - 13s 110us/step - loss: 0.0245
Epoch 622/1000
116351/116351 [==============================] - 13s 110us/s

116351/116351 [==============================] - 13s 110us/step - loss: 0.0250
Epoch 787/1000
116351/116351 [==============================] - 13s 110us/step - loss: 0.0242
Epoch 788/1000
116351/116351 [==============================] - 13s 110us/step - loss: 0.0258
Epoch 789/1000
116351/116351 [==============================] - 13s 110us/step - loss: 0.0247
Epoch 790/1000
116351/116351 [==============================] - 13s 110us/step - loss: 0.0242
Epoch 791/1000
116351/116351 [==============================] - 13s 109us/step - loss: 0.0242
Epoch 792/1000
116351/116351 [==============================] - 13s 110us/step - loss: 0.0248
Epoch 793/1000
116351/116351 [==============================] - 13s 110us/step - loss: 0.0243
Epoch 794/1000
116351/116351 [==============================] - 13s 110us/step - loss: 0.0239
Epoch 795/1000
116351/116351 [==============================] - 13s 110us/step - loss: 0.0245
Epoch 796/1000
116351/116351 [==============================] - 13s 110us/s

116351/116351 [==============================] - 13s 110us/step - loss: 0.0243
Epoch 961/1000
116351/116351 [==============================] - 13s 110us/step - loss: 0.0244
Epoch 962/1000
116351/116351 [==============================] - 13s 110us/step - loss: 0.0242
Epoch 963/1000
116351/116351 [==============================] - 13s 110us/step - loss: 0.0243
Epoch 964/1000
116351/116351 [==============================] - 13s 110us/step - loss: 0.0240
Epoch 965/1000
116351/116351 [==============================] - 13s 110us/step - loss: 0.0249
Epoch 966/1000
116351/116351 [==============================] - 13s 110us/step - loss: 0.0258
Epoch 967/1000
116351/116351 [==============================] - 13s 109us/step - loss: 0.0237
Epoch 968/1000
116351/116351 [==============================] - 13s 110us/step - loss: 0.0242
Epoch 969/1000
116351/116351 [==============================] - 13s 110us/step - loss: 0.0238
Epoch 970/1000
116351/116351 [==============================] - 13s 110us/s

In [16]:
# Save a model you have trained
estimator_A.model.save('Models/Two_stage/trained_model.h5')

In [17]:
# Save a model you have trained
estimator_A.model.save('Models/Two_stage/trained_model0.h5')

In [10]:
# fit the model with the first half of dataset
for i in range(2,3):
    estimator_A = load_model('Models/Two_stage/trained_model' +str(i-1)+ '.h5')
    ftr , target = data_reading(i)
    X = ftr
    y = target
    ftr = []
    target = []
    print("^^^^^^^^^^^^^^^^^^^^^ Training on mixed" + str(i) + "^^^^^^^^^^^^^^^^^^^^^")
    tic = time.time()
    estimator_A.fit(X, y, epochs=1000, batch_size=150)
    toc = time.time()
    print("^^^^^^^^^^^^^^^^^ Time of Training on mixed" + str(i) + "=", toc - tic)
    estimator_A.model.save('Models/Two_stage/trained_model' +str(i)+ '.h5')

^^^^^^^^^^^^^^^^^^^^^ Training on mixed2^^^^^^^^^^^^^^^^^^^^^
Epoch 1/1000
120729/120729 [==============================] - 15s 124us/step - loss: 0.1186
Epoch 2/1000
120729/120729 [==============================] - 14s 114us/step - loss: 0.1070
Epoch 3/1000
120729/120729 [==============================] - 14s 114us/step - loss: 0.1018
Epoch 4/1000
120729/120729 [==============================] - 14s 114us/step - loss: 0.0978
Epoch 5/1000
120729/120729 [==============================] - 14s 114us/step - loss: 0.0947
Epoch 6/1000
120729/120729 [==============================] - 14s 115us/step - loss: 0.0918
Epoch 7/1000
120729/120729 [==============================] - 14s 114us/step - loss: 0.0894
Epoch 8/1000
120729/120729 [==============================] - 14s 114us/step - loss: 0.0872
Epoch 9/1000
120729/120729 [==============================] - 14s 114us/step - loss: 0.0850
Epoch 10/1000
120729/120729 [==============================] - 14s 114us/step - loss: 0.0832
Epoch 11/1000
120

120729/120729 [==============================] - 13s 109us/step - loss: 0.0394
Epoch 176/1000
120729/120729 [==============================] - 13s 110us/step - loss: 0.0400
Epoch 177/1000
120729/120729 [==============================] - 13s 109us/step - loss: 0.0407
Epoch 178/1000
120729/120729 [==============================] - 13s 110us/step - loss: 0.0398
Epoch 179/1000
120729/120729 [==============================] - 13s 109us/step - loss: 0.0396
Epoch 180/1000
120729/120729 [==============================] - 13s 109us/step - loss: 0.0395
Epoch 181/1000
120729/120729 [==============================] - 13s 109us/step - loss: 0.0396
Epoch 182/1000
120729/120729 [==============================] - 13s 109us/step - loss: 0.0391
Epoch 183/1000
120729/120729 [==============================] - 13s 109us/step - loss: 0.0390
Epoch 184/1000
120729/120729 [==============================] - 13s 110us/step - loss: 0.0391
Epoch 185/1000
120729/120729 [==============================] - 13s 110us/s

120729/120729 [==============================] - 13s 109us/step - loss: 0.0358
Epoch 350/1000
120729/120729 [==============================] - 13s 109us/step - loss: 0.0359
Epoch 351/1000
120729/120729 [==============================] - 13s 109us/step - loss: 0.0357
Epoch 352/1000
120729/120729 [==============================] - 13s 109us/step - loss: 0.0356
Epoch 353/1000
120729/120729 [==============================] - 13s 109us/step - loss: 0.0363
Epoch 354/1000
120729/120729 [==============================] - 13s 109us/step - loss: 0.0358
Epoch 355/1000
120729/120729 [==============================] - 13s 110us/step - loss: 0.0358
Epoch 356/1000
120729/120729 [==============================] - 13s 109us/step - loss: 0.0351
Epoch 357/1000
120729/120729 [==============================] - 13s 109us/step - loss: 0.0357
Epoch 358/1000
120729/120729 [==============================] - 13s 109us/step - loss: 0.0359
Epoch 359/1000
120729/120729 [==============================] - 13s 109us/s

120729/120729 [==============================] - 13s 109us/step - loss: 0.0336
Epoch 524/1000
120729/120729 [==============================] - 13s 109us/step - loss: 0.0344
Epoch 525/1000
120729/120729 [==============================] - 13s 109us/step - loss: 0.0339
Epoch 526/1000
120729/120729 [==============================] - 13s 109us/step - loss: 0.0337
Epoch 527/1000
120729/120729 [==============================] - 13s 109us/step - loss: 0.0339
Epoch 528/1000
120729/120729 [==============================] - 13s 109us/step - loss: 0.0341
Epoch 529/1000
120729/120729 [==============================] - 13s 109us/step - loss: 0.0367
Epoch 530/1000
120729/120729 [==============================] - 13s 109us/step - loss: 0.0341
Epoch 531/1000
120729/120729 [==============================] - 13s 109us/step - loss: 0.0341
Epoch 532/1000
120729/120729 [==============================] - 13s 109us/step - loss: 0.0334
Epoch 533/1000
120729/120729 [==============================] - 13s 109us/s

120729/120729 [==============================] - 14s 112us/step - loss: 0.0329
Epoch 698/1000
120729/120729 [==============================] - 14s 113us/step - loss: 0.0326
Epoch 699/1000
120729/120729 [==============================] - 14s 113us/step - loss: 0.0329
Epoch 700/1000
120729/120729 [==============================] - 14s 113us/step - loss: 0.0334
Epoch 701/1000
120729/120729 [==============================] - 14s 113us/step - loss: 0.0328
Epoch 702/1000
120729/120729 [==============================] - 14s 113us/step - loss: 0.0332
Epoch 703/1000
120729/120729 [==============================] - 14s 113us/step - loss: 0.0323
Epoch 704/1000
120729/120729 [==============================] - 14s 113us/step - loss: 0.0325
Epoch 705/1000
120729/120729 [==============================] - 14s 113us/step - loss: 0.0332
Epoch 706/1000
120729/120729 [==============================] - 14s 113us/step - loss: 0.0329
Epoch 707/1000
120729/120729 [==============================] - 14s 113us/s

120729/120729 [==============================] - 14s 113us/step - loss: 0.0332
Epoch 872/1000
120729/120729 [==============================] - 14s 113us/step - loss: 0.0319
Epoch 873/1000
120729/120729 [==============================] - 14s 113us/step - loss: 0.0316
Epoch 874/1000
120729/120729 [==============================] - 14s 112us/step - loss: 0.0324
Epoch 875/1000
120729/120729 [==============================] - 14s 112us/step - loss: 0.0334
Epoch 876/1000
120729/120729 [==============================] - 14s 113us/step - loss: 0.0339
Epoch 877/1000
120729/120729 [==============================] - 14s 112us/step - loss: 0.0326
Epoch 878/1000
120729/120729 [==============================] - 14s 113us/step - loss: 0.0320
Epoch 879/1000
120729/120729 [==============================] - 14s 113us/step - loss: 0.0316
Epoch 880/1000
120729/120729 [==============================] - 14s 117us/step - loss: 0.0320
Epoch 881/1000
120729/120729 [==============================] - 14s 113us/s

/home/speech/anaconda3/envs/SGL/lib/python3.6/site-packages/keras/engine/sequential.py:110: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '


In [ ]:
I=0
def da(batch_size, data_len):
    global I
    while True:             #this line is just because keras needs infinite generators
        for I in range(0,data_len-batch_size,batch_size): 
            h5f = h5py.File('ftr_scaled.hdf5','r')
            Data1 = h5f['ftr_scaled'][I:I+batch_size]
            h5f.close()
            h5f = h5py.File('single_dataset_log_16.hdf5','r')
            Data2 = h5f['single_dataset_log_16'][I:I+batch_size]
            h5f.close()
            yield(Data1, Data2)

In [ ]:
# model.fit_generator(da(batch_size, target.shape[0]), steps_per_epoch=steps,epochs=50, verbose=1, callbacks = lr_callbacks)

In [30]:
Steps = 0
def data_reader(path, batch_len):
    global Steps
    while True:
        for i in range(4):
            temp = open( path + "Mixed" +str(i) + ".p", "rb")
            ftr = pickle.load( temp  ) .T
            temp.close()
            #contarins target(clean) log power features.
            temp = open( path + "clean" +str(i) + ".p", "rb")
            target = pickle.load( temp ).T
            temp.close()
            # create numpy arrays of input data
            # and labels, from each line in the file
            Steps = np.floor(ftr.shape[0]/batch_len)
            for batch in range(0, ftr.shape[0]-batch_len, batch_len):
                print('path=', path + "Mixed" +str(i) + ".p")
                print('path=', path + "clean" +str(i) + ".p")
                print('batch =', batch)
                X = ftr[batch:batch + batch_len]
                y = target[batch:batch + batch_len]
                yield (X, y)

In [31]:
path = 'TIMIT/Organized/concatenated/'
batch_len = 128
dr = data_reader(path, batch_len)

In [32]:
# estimator_A = KerasRegressor(build_fn=DNN_A, epochs=2, batch_size=150, verbose=1)
# kfold = KFold(n_splits=5)
X, y = data_reading(0)
model_A = Sequential()
model_A.add(Dense(h_A[0], input_dim = X.shape[1], kernel_initializer='normal', activation='relu'))
model_A.add(Dense(h_A[1], kernel_initializer='normal', activation='relu'))
model_A.add(Dense(h_A[2], kernel_initializer='normal', activation='relu'))
model_A.add(Dense(y.shape[1], kernel_initializer='normal'))
# Compile model
model_A.compile(loss='mean_squared_error', optimizer='adam')
X = []
y = []

In [29]:
print(next(len(dr)))

TypeError: object of type 'generator' has no len()

In [33]:
# path = 'TIMIT/Organized/concatenated/'
model_A.fit_generator(generator=dr,
                          steps_per_epoch=10, 
                          epochs=3,
#                           batch_size=150,
                          shuffle=True,
                          verbose=1)

Epoch 1/3
path= TIMIT/Organized/concatenated/Mixed0.p
path= TIMIT/Organized/concatenated/clean0.p
batch = 0
path= TIMIT/Organized/concatenated/Mixed0.p
path= TIMIT/Organized/concatenated/clean0.p
batch = 128
path= TIMIT/Organized/concatenated/Mixed0.p
path= TIMIT/Organized/concatenated/clean0.p
batch = 256
path= TIMIT/Organized/concatenated/Mixed0.p
path= TIMIT/Organized/concatenated/clean0.p
batch = 384
path= TIMIT/Organized/concatenated/Mixed0.p
path= TIMIT/Organized/concatenated/clean0.p
batch = 512
path= TIMIT/Organized/concatenated/Mixed0.p
path= TIMIT/Organized/concatenated/clean0.p
batch = 640
path= TIMIT/Organized/concatenated/Mixed0.p
path= TIMIT/Organized/concatenated/clean0.p
batch = 768
path= TIMIT/Organized/concatenated/Mixed0.p
path= TIMIT/Organized/concatenated/clean0.p
batch = 896
path= TIMIT/Organized/concatenated/Mixed0.p
path= TIMIT/Organized/concatenated/clean0.p
batch = 1024
path= TIMIT/Organized/concatenated/Mixed0.p
path= TIMIT/Organized/concatenated/clean0.p
bat

path= TIMIT/Organized/concatenated/clean0.p
batch = 5120


In [13]:
class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=128, dim=(1025,1), n_channels=1, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
#         self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            temp0 = open( "TIMIT/Organized/concatenated/Mixed" + ID + ".p", "rb")
            X[i] = np.pickle.load(temp0).T

            # Store target
            temp1 = open( "TIMIT/Organized/concatenated/clean" + ID + ".p", "rb")
            y[i] = np.pickle.load(temp1).T
            yield X, y

In [14]:
partition = {}
partition['train'] = ['0', '1', '2']
labels = ['0', '1', '2']

# Parameters
params = {
            'dim': (1025,1),
            'batch_size': 128,
#           'n_classes': 6,
            'n_channels': 1,
            'shuffle': True}

training_generator = DataGenerator(partition['train'], labels, **params)

In [12]:
print(training_generator)

In [21]:
training_generator

In [15]:
ftr, target = data_reading(0)
model_A = Sequential()
model_A.add(Dense(h_A[0], input_dim = ftr.shape[1], kernel_initializer='normal', activation='relu'))
model_A.add(Dense(h_A[1], kernel_initializer='normal', activation='relu'))
model_A.add(Dense(h_A[2], kernel_initializer='normal', activation='relu'))
model_A.add(Dense(target.shape[1], kernel_initializer='uniform'))
# Compile model
model_A.compile(loss='mean_squared_error', optimizer='adam')
ftr = []
target = []

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [16]:
# Train model on dataset
model_A.fit_generator(generator=training_generator,
                      steps_per_epoch=training_generator.__len__(),
                      use_multiprocessing=True,
                      workers=6,
                      epochs=20,
                      initial_epoch=1)

Instructions for updating:
Use tf.cast instead.
Epoch 2/20


IndexError: list index out of range

In [ ]:
def get_input(path):
    
    temp = open( path, "rb")
    X = np.pickle.load(temp).T    
    return(X)

def get_output(path):
    
    temp = open( path, "rb")
    y = np.pickle.load(temp).T    
    return(y)

def image_generator(files,label_file, batch_size = 128):
    i = 0
    while True:
        # Select files (paths/indices) for the batch
        file_path = "TIMIT/Organized/concatenated/Mixed" + str(i) + ".p"
#         batch_paths = np.random.choice(a = files, 
#                                      size = batch_size)
        batch_input = []
        batch_output = [] 

        # Read in each input, perform preprocessing and get labels
        for input_path in batch_paths:
          input = get_input(input_path )
          output = get_output(input_path,label_file=label_file )

          input = preprocess_input(image=input)
          batch_input += [ input ]
          batch_output += [ output ]
        # Return a tuple of (input,output) to feed the network
        batch_x = np.array( batch_input )
        batch_y = np.array( batch_output )
        i = i + 1
        
        yield( batch_x, batch_y )

In [6]:
batch_paths = np.random.choice(a = [1, 2, 3, 4], size = 2)

In [7]:
batch_paths

array([4, 2])

In [109]:
class MY_Generator2(Sequence):

    def __init__(self, filenames, labels, batch_size):
        self.image_filenames, self.labels = image_filenames, labels
        self.batch_size = batch_size

    def __len__(self):
        return np.ceil(len(self.filenames) / float(self.batch_size))

    def __getitem__(self, idx):
        batch_x = self.filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]

        return batch_x, batch_y

In [23]:
temp = open( "TIMIT/Organized/concatenated/Mixed" +str(2) + ".p", "rb")

In [8]:
ftr , target = data_reading(2)

In [9]:
ftr = []
target = []

In [28]:
def data_reading(i):
    temp = gzip.open( "TIMIT/Organized/concatenated/Mixed" +str(i) + ".p", "rb")
    ftr = pickle.load( temp  ) .T
    temp.close()
    #contarins target(clean) log power features.
    temp = open( "TIMIT/Organized/concatenated/clean" +str(i) + ".p", "rb")
    target = pickle.load( temp ).T
    temp.close()
    return ftr, target

NameError: name 'gzip' is not defined

In [45]:
# generating the second set of dataset
for i in range(2,4):
    estimator_A = load_model('Models/Two_stage/trained_model1.h5')
    ftr , target = data_reading(i)
    X_sec = ftr
#     y_sec = target

    prediction = estimator_A.predict(X_sec)
    pickle.dump( prediction, open( "TIMIT/Two_stage_sec_set/new"+str(i)+".p", "wb" ))
    ftr = []
    target = []

In [ ]:
# End of training first stage

In [284]:
estimator_A = load_model('Models/Two_stage/trained_model0.h5')


In [305]:
X_test, y_test = data_reading(0)

In [306]:
prediction = estimator_A.predict(X_test)

In [307]:
angle = phase_reading(0)

In [308]:
y_test.shape

(116351, 2050)

In [325]:
n0_test = 100000
n1_test = 100300

In [326]:
speaker1 = reconstruct(prediction[n0_test:n1_test, 0:1025], angle[n0_test:n1_test])
label = reconstruct(y_test[n0_test:n1_test, 0:1025], angle[n0_test:n1_test])
X_recon = reconstruct(X_test[n0_test:n1_test], angle[n0_test:n1_test])

In [327]:
prediction[n0_test:n1_test, 0:1025].shape

(300, 1025)

In [328]:
speaker1.shape

(153088,)

In [329]:
label.shape

(153088,)

In [330]:
sdr, sir, sar, perm = separation.bss_eval_sources(label, speaker1, compute_permutation=True)

In [331]:
sdr1, sir1, sar1, perm1 = separation.bss_eval_sources(label, X_recon, compute_permutation=True)

In [332]:
sdr

array([10.35246227])

In [333]:
sdr1

array([-2.97054587])

In [334]:
st = stoi(label, speaker1, 16000, extended=False)
st1 = stoi(label, X_recon, 16000, extended=False)

In [335]:
st

0.7593615683762284

In [336]:
st1

0.5835074155285023

In [337]:
sd.play(speaker1*10, 16000)

In [322]:
sd.play(label*10, 16000)

In [323]:
sd.play(X_recon*10, 16000)

In [22]:
# from keras.callbacks import ModelCheckpoint
# # define the checkpoint
# filepath = "model.h5"
# checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
# callbacks_list = [checkpoint]

In [23]:
# # fit the model
# estimator_A.fit(X[300:600], y[300:600], epochs=10, batch_size=150, callbacks=callbacks_list)

Epoch 1/10
300/300 [==============================] - 1s 3ms/step - loss: 1.6880

Epoch 00001: loss improved from inf to 1.68803, saving model to model.h5
Epoch 2/10
300/300 [==============================] - 0s 555us/step - loss: 0.2899

Epoch 00002: loss improved from 1.68803 to 0.28990, saving model to model.h5
Epoch 3/10
300/300 [==============================] - 0s 540us/step - loss: 0.2072

Epoch 00003: loss improved from 0.28990 to 0.20718, saving model to model.h5
Epoch 4/10
300/300 [==============================] - 0s 625us/step - loss: 0.1929

Epoch 00004: loss improved from 0.20718 to 0.19287, saving model to model.h5
Epoch 5/10
300/300 [==============================] - 0s 514us/step - loss: 0.1905

Epoch 00005: loss improved from 0.19287 to 0.19050, saving model to model.h5
Epoch 6/10
300/300 [==============================] - 0s 508us/step - loss: 0.1889

Epoch 00006: loss improved from 0.19050 to 0.18888, saving model to model.h5
Epoch 7/10
300/300 [====================

In [25]:
# fit the model
# estimator_A.fit(X[300:600], y[300:600])

Epoch 1/10
300/300 [==============================] - 1s 3ms/step - loss: 1.6880
Epoch 2/10
300/300 [==============================] - 0s 542us/step - loss: 0.2993
Epoch 3/10
300/300 [==============================] - 0s 567us/step - loss: 0.2086
Epoch 4/10
300/300 [==============================] - 0s 570us/step - loss: 0.1932
Epoch 5/10
300/300 [==============================] - 0s 592us/step - loss: 0.1905
Epoch 6/10
300/300 [==============================] - 0s 577us/step - loss: 0.1889
Epoch 7/10
300/300 [==============================] - 0s 578us/step - loss: 0.1864
Epoch 8/10
300/300 [==============================] - 0s 586us/step - loss: 0.1829
Epoch 9/10
300/300 [==============================] - 0s 576us/step - loss: 0.1787
Epoch 10/10
300/300 [==============================] - 0s 639us/step - loss: 0.1741
